# Fixed Precision Encoding

Author: 
- Théo Ryffel - [Twitter](http://twitter.com/theoryffel) - [GitHub](https://github.com/LaRiffle)

### [Video] Introduction to FixedPrecision vs FloatPrecision
Script:
- This section is about fixed precision encoding. Fixed point precision encoding is the counterpart of floating point precision encoding. To better understand what is fixed point precision (or fixed precision in short) and why we use it, let’s start by understanding float precision, the most commonly used encoding.
- In statistics, we often deal with real numbers, not integers! This is very helpful for math analysis, especially to be able to use concepts like differentiation that we use intensively during back-propagation when we train a neural network.
- But computers are only made of zeros and ones! Therefore we need a way to represent all numbers in terms of 0 and 1, namely we need to find a proper encoding.
- So first, we are always considering decimal numbers. If you think of $\pi$, we can't use it directly, but only a decimal approximation, which might be sufficient for many computations.
- The most common way to encode decimal numbers is to encode them on 32 bits, using the float precision encoding. The idea of floating precision is close to the notion of scientific notation : we store separately the most significant numbers (that we call the mantissa), and the exponent, that is a power of 10 (or a power of 2 in practice), and we multiply them together to reconstruct the original number. You can think of the exponent as the position of the point in the decimal notation.
- Let's take an example, if I wanted to store 3.14, I would store separately the digits 314 and the position of the point.
- It makes it very convenient to store with high fidelity very big and very small numbers: you only need to say that the exponent is very big or very small. This encoding is therefore widely used in practice.
- However, when you compute the addition of two floating point numbers with different exponents, you need to convert them to adopt the same exponent, which can be complicated when those numbers are private as in secure multi-computation.
- Therefore, we will use fixed precision, that we will explore more in depth just now!

### Floating point and fixed point arithmetic

In the video above, we've been through the concept of floating point numbers. A schematic way of understanding this encoding is to compare it to the scientific notation.

Let's take an example, like $123.456$. The scientific notation is $1.23456 \times 10^2$, where $1.23456$ is stored on the mantissa and $2$ in the exponent. Depending on the size of the mantissa, we might need to drop the least significant digits and store for example $1.23 \times 10^2$. In practice, we will use powers of $2$ in the float encoding, but for now we can stick to $10$ for better readability.

When we want to add 2 numbers, $123.456$ and $7.3$, which are encoded in such a way that the mantissa only stores the 3 most significant digits, then we can write them $1.23 \times 10^2$ and $7.3 \times 10^0$. To be able to add them, we will rewrite the numbers to express them with the same exponent, usually the largest. In our example, we will hence rewrite the addition: $1.23 \times 10^2 + 0.07 \times 10^2$. Note that we have lost some precision.

You can find more about floating point precision [here](https://www.doc.ic.ac.uk/~eedwards/compsys/float/).

Adapting the exponents can be costly when dealing with private numbers, where the mantissa and the exponent are not available in clear text. Therefore, in many implementations of secure multi-party computation, we use fixed point precision encoding, which as you might guess is actually much simpler!

Basically, it consists of taking a fixed public exponent for all the numbers involved in the computation. Therefore, we usually consider separately an integer value (for example a `LongTensor` if you're using PyTorch or a `np.int64` for Numpy users), and the fixed exponent. For example, we could set the precision to $10^{-6}$, and a number like $123.456$ would be encoded as $123456000 \times 10^{-6}$.

This makes addition very easy because you don't have any rescaling to compute: you just have to sum the integers values and keep the exponent unchanged. One limitation however of fixed precision is that the range of values you can store is much more limited, you can’t store numbers smaller than $10^{-6}$ in the example above. In addition, the smaller the precision (like if you reduce from 6 to 2), the bigger the rounding error of your encoding. In practice, as we are doing classic statistics and machine learning, such encoding is good enough to achieve accuracies close to floating point precision.

There is one subtlety however, which has to do with multiplication. Let's consider for example the multiplication of $1.2$ and $2.0$ encoded with a decimal precision of 2, so $120 \times 10^{-2}$ and $200 \times 10^{-2}$. If we multiply the integer part, and keep one of the encodings, we end up with $24000 \times 10^{-2}$ so $240$ instead of $2.4$. We can either increase the precision from 2 to 4, but in the fixed precision setting we really want the precision to remain constant, or we can truncate the integer part, by removing the 2 least significant digits which correspond to the current precision of 2: $(24000 // 10^2) \times 10^{-2} = 240 \times 10^{-2} = 2.4$.

So far, we have been considering the decimal precision, i.e with base 10, but in practice it's more suitable to use the bit precision, i.e with base 2, because the size of the mantissa is expressed in number of bits. $1.5$ encoded in a bit precision of 3 would be $12 \times 2^{-3}$. We will use both in the rest of the course.

Ok this was a lot of theory, let's practice now!

### Implementing a fixed precision encoder and decoder

We will now implement this fixed precision mechanism, to be able to better understand it. We will consider that the `base` and the `precision` are public parameters.

Let's start by building the encoder and the decoder.

Note:
- You should round values to the nearest integer.
- We used typing annotations, which can be useful hints in same cases 😉

In [ ]:
# we import the answer checker
from quiz import test_encode, test_decode

In [ ]:
base = 10
precision = 3

def encode(x: float, base: int, precision: int) -> int: 
    # complete the function to encode values in fixed precision
    pass

You can use the following cell to check that your function behaves properly. It will run some tests and return an error if it encounters an unexpected result.

In [ ]:
# run to check your function
test_encode.check(encode)

For each coding exercise, you will also have a hint, and the solution if you are blocked. Don't forget do use it if you're blocked!

In [ ]:
# Uncomment the line below to see a hint
# test_encode.hint

In [ ]:
# Uncomment the line below to see the solution
# test_encode.solution

Let's do the decoder now!

In [ ]:
def decode(x: float, base: int, precision: int) -> float:
    # complete the function to decode fixed precision values
    pass

In [ ]:
# run to check your function
test_decode.check(decode)

In [ ]:
# Uncomment the line below to see a hint
# test_decode.hint
# Uncomment the line below to see the solution
# test_decode.solution

Let's play a bit with our functions

In [ ]:
base = 10
precision = 3

assert encode(1.2, base, precision) == 1200
assert encode(1.0008, base, precision) == 1001 # Values with decimal after the 3rd are rounded

assert decode(1200, base, precision) == 1.2
assert decode(1001, base, precision) == 1.001 # Hence the decoding is only approximate here


# Let's change the base
base = 2
precision = 5

assert encode(1.2, base, precision) == 38

print(decode(38, base, precision))
# Note, you can observe the rounding error here, as it doesn't decode to 1.2 exactly

### Operating on fixed precision values

We should now be well equipped to perform additions! Let's check.

In [ ]:
import numpy as np
base = 2
precision = 10

x, y = 1.2, 1.1

# _fp stands for "fixed precision"
x_fp, y_fp = encode(x, base, precision), encode(y, base, precision)

print("x_fp, y_fp :", x_fp, y_fp)

z_fp = x_fp + y_fp

print("z_fp :", z_fp)

z = decode(z_fp, base, precision)

diff = z - (x + y)
assert np.abs(diff) <= base ** -precision # we check that the rounding error is "small"

print('computed sum:', z, 'expected:', x + y)

Next, to compute multiplication, we need to add the truncation mechanism. Let's implement it! 

Write the code for the `truncation` and the `mul` functions.

In [ ]:
base = 10
precision = 3

def truncate(x: int, base: int, precision: int) -> int:
    # Write your code here
    pass
    
def mul(x: int, y: int, base: int, precision: int) -> int:
    # Write your code here
    pass

In [ ]:
# we import the answer checker
from quiz import test_fp_truncate, test_fp_mul

test_fp_truncate.check(truncate)

test_fp_mul.check(mul)

In [ ]:
# Uncomment the line below to see a hint
# test_fp_truncate.hint
# Uncomment the line below to see the solution
# test_fp_truncate.solution

# Uncomment the line below to see a hint
# test_fp_mul.hint
# Uncomment the line below to see the solution
# test_fp_mul.solution

In [ ]:
x_fp = encode(1.2, base, precision)
y_fp = encode(2.0, base, precision)

print("x_fp, y_fp :", x_fp, y_fp)

z_fp = mul(x_fp, y_fp, base, precision)

print("z_fp :", z_fp)
z = decode(z_fp, base, precision)

assert z == 2.4 # It's working just fine!

To implement matrix multiplication in fixed precision, you could follow exactly the same steps, you just have to replace `*` with `@`, the matrix multiplication operator.

Excellent! At this point we are well equipped to operate on fixed precision numbers. Now, let's go a bit further and instead of considering fixed precision encoding of public numbers, let's consider fixed precision encoding of secret shared numbers, we're doing SMC after all!

### Application to secret sharing

As we recall in SMC, shares are not float values but integers in a sufficiently big space, for example in $\mathbb{Z}_{2^{n}}$, the space of the numbers encoded on $n$ bits, with typically $n=32$. We will apply the mechanisms we have just seen to secret shared numbers.

In practice, we assume that a private value is first encoded to fixed precision before being secret shared, and we want to study how fixed precision impacts basic operations like:
- Addition
- Division by a public constant

For addition, the great news is that there is nothing to change! In the computation $x + y$, each party still sum his shares of $x$ and $y$.

For division by a public constant, which is a very common operation that you perform for example when you compute the mean $\frac{1}{n} \sum_{i=1}^n x_i$, things get a bit more complicated!

Indeed, the most basic idea would be to ask each party to compute the division on its share, so that when the shares are summed up, the result will be the original result divided by the public constant. 

Let's use examples to see how this naive method behaves:

##### A working public division

We consider shares to be in $\mathbb{Z}_{2^{4}}$ to simplify the computation. $\mathbb{Z}_{2^{4}}$ represents all the integers in the range $[-8, 7 ]$. Indeed, $2^{4} = 16$ and we consider signed numbers, so instead of the range $[0, 15]$, we use $[-8, 7]$. We will get back to this convention a bit later.

Let's consider $x = 4$, shared between Alice and Bob as such: $[x] = (x_{alice}, x_{bob}) = (-2, 6)$. If each party computes the division by 2 on its share, we obtain $(-1, 3)$ which reconstructs to $2$, and which is the correct result.

But sometimes, we are not as lucky...

##### A non-working public division

Let's now consider $x = -4$, shared as such: $[x] = (x_{alice}, x_{bob}) = (6, 6)$. This sharing is correct since 6 + 6 = 12 which maps to 12 - 16 = -4 in $\mathbb{Z}_{2^{4}}$. We say that there is a _wrap around_, as we exceeded the maximum value of our space, 7. Dividing the shares by 2 like before would lead to a sharing (3, 3), which reconstructs to $6 \neq -4/2$. It's failing!!

So what happened? Actually the shared result (3, 3) is still valid, but no longer in $\mathbb{Z}_{2^{4}}$ but in the space of values divided by 2, namely $\mathbb{Z}_{2^{3}}$. Indeed, $\mathbb{Z}_{2^{3}}$ represents the range $[-4, 3]$, so 3 + 3 = 6 should map to 6 - 8 = -2 in $\mathbb{Z}_{2^{3}}$. However, we want to keep the same space for all the shares that we are using, so we need to find a workaround.

The reason why it failed is because of the _wrap around_, which happens when we sum the shares of $x$ but not when we sum the shares of $x / 2$. It is not possible for any of the parties holding a single share of $x$ to know if there is a wrap around $\theta_x$, because it would reveal information about the other parties shares. However, they can privately compute the wrap around and obtain a secret shared $[\theta_x]$. Computing such $[\theta_x]$ is non-trivial, and we will leave as a black box the function that computes it.

Having access to $[\theta_x]$ helps to compute privately and correctly $x / k$ in $\mathbb{Z}_{2^{n}}$ using the following formula:
$$ [x / k] = [x] / k - [\theta_x] \cdot 2^{n} / k$$
This means that each party divides naively its shares, and they jointly correct the result if there was a wrap around (i.e. if $\theta_x \neq 0$).

You can check on the previous non-working example that this formula works. Indeed, we have $\theta_x = 1$ and $2^{n} / k = 16 / 2 = 8$. If we assume $[\theta_x]= (1, 0)$, then the shares of the result would not be $(3, 3)$ but $(-5, 3)$, which sums up to $-2$. Et voilà!





### Extra note: numerical errors in division

You've been through the most intense part, and before we switch to multiplication and truncation, we just make a note on the numerical errors when using small fields. In the examples above, we have been playing in $\mathbb{Z}_{2^{4}}$.

If we consider $x = 4$ to be shared in $\mathbb{Z}_{2^{4}}$ as such: $[x] = (1, 3)$, and that we apply division per 2, even with our method involving $\theta_x$ that is 0 here, we will end up with shares (0, 1) (because 1 // 2 = 0 and 3 // 2 = 1), that sum up to $1 \neq 4/2$.

The rounding error here causes the result to be completely incorrect, but for bigger spaces like $\mathbb{Z}_{2^{32}}$ that we usually consider, this error will be less important. Just remember that it is here, and that it can impact marginally your computation.

### Multiplication with secret shared fixed precision numbers

We have seen previously that multiplication with fixed precision values requires to use truncation. Since the parameters of the fixed precision (the base and the precision) are public, we can observe that truncation is just a public division, which we now have learned to do correctly!

For example, for a base of 10 and a precision of 3, truncation is just the division by 1000. For base 2 and precision 2, it is the division by 4. Easy, isn't it?

Let's write the code to do this private multiplication using truncation. We will assume that we can use the following functions:
- `encode(x: float) -> int`, that encode real numbers to fixed precision
- `decode(x: int) -> float`, that convert fixed precision numbers back to floats.
- `secret_share(x: int) -> List[int]`, that builds shares of a fixed precision number
- `decrypt(shares: List[int]) -> int`, that reconstructs a fixed precision number from its shares
- `spdz_mul(x_shares: List[int], y_shares: List[int]) -> List[int]`, that given $x$ and $y$ shared integers, returns the shares of $x * y$  
- `wrap_around(shares: List[int]) -> List[int]`, that given some shares return the number of wrap arounds in a secret shared fashion

We have removed the `base`, `precision` and `bits` arguments in the functions, we will assume that they are globally set and equal to `10`, `2` and `32` respectively for the checks.

You should follow the implementation of the previous section, by defining a `mul` function that will call a `truncate` function.

In [ ]:
from quiz import encode, decode, secret_share, decrypt, spdz_mul, wrap_around
from typing import List

base = 10
precision = 3
bits = 32 # on how many bits we encode our fixed precision values and shares

def truncate(x_shares: List[int]) -> List[int]:
    # Write your code here
    pass

def mul(x_shares: List[int], y_shares: List[int]) -> List[int]:
    # Write your code here
    pass

In [ ]:
# we import the answer checker
from quiz import test_truncate, test_mul

test_truncate.check(truncate)

In [ ]:
# Uncomment the line below to see a hint
# test_truncate.hint
# Uncomment the line below to see the solution
# test_truncate.solution

In [ ]:
test_mul.check(mul)

In [ ]:
# Uncomment the line below to see a hint
# test_mul.hint
# Uncomment the line below to see the solution
# test_mul.solution

Here you are! We have now explored most of the subtleties of combining fixed precision with secret sharing. All of this complexity is completely hidden when you are using PySyft, as it takes care of this for you.

Here is the equivalent code you would write with PySyft:

In [ ]:
import syft as sy
from sympc.session import Session
from sympc.tensor import MPCTensor

alice = sy.VirtualMachine(name="Alice").get_client()
bob = sy.VirtualMachine(name="Bob").get_client()

session = Session(parties=(alice, bob))
Session.setup_mpc(session)

x_secret = torch.Tensor([1.2])
y_secret = torch.Tensor([2.1])

x = MPCTensor(secret=x_secret, session=session)
y = MPCTensor(secret=y_secret, session=session)

result = (x * y).reconstruct()

Let's now take a little tour in theory! In the next part, you will be discovering more about the security guarantees that you can have as a user of secure multi-party computation.